# Emisiones de Gases Efecto Invernadero
## Datos obtenidos desde Ministerio del Medio Ambiente

[Base de Datos Inventarios Regionales de GEI serie 1990-2018](https://snichile.mma.gob.cl/wp-content/uploads/2021/06/inventarios_regionales_1990-2018.zip)

https://snichile.mma.gob.cl/wp-content/uploads/2021/06/Informe-Inventarios-Regionales-serie-1990-2018.pdf

In [ ]:
import pandas as pd
from pathlib import Path
import altair as alt

In [ ]:
# Ruta datos GEI
DATOS_GEI = Path("../data/GEI/")

In [ ]:
# Codigo regiones macrozona asociado al archivo xlx
zona = ['11BIO','09MAU','10NUB', '08OHI']

datos_zona = list(filter(lambda f: f.name.split("_")[2] in zona, DATOS_GEI.glob("*.xlsx")))
datos_zona

### Plot ejemplo con una región

In [ ]:
# Cargar un archivo con datos
gei_xls = pd.ExcelFile(datos_zona[0])
gei_data = pd.read_excel(gei_xls, sheet_name=["GPC alcances serie CO2eq"],
                         index_col=[0,1],header=[0],skiprows=52,nrows=11)

In [ ]:
# Seleccionar hoja de excel `GPC alcances serie CO2eq`
gei_data_f1 = gei_data["GPC alcances serie CO2eq"].iloc[1:,:-4]

# clasificaciones Gases de Efecto Invernado a Escala Comunitaria GPC
class_gpc = ["Energía estacionaria","Transporte","Residuos","IPPU","Agricultura","UTCUTS"] 
gei_data_f2 = gei_data_f1.loc[gei_data_f1["Clasificación GPC"].isin(class_gpc)]

In [ ]:
gei_data_f2.head()

In [ ]:
gei_tidy = gei_data_f2.melt(value_name="kt CO2 eq",var_name="Año",id_vars=["Clasificación GPC"],
        value_vars=gei_data_f2.columns[1:])
gei_tidy.head()

In [ ]:
bars = alt.Chart(gei_tidy).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    y = alt.Y('kt CO2 eq:Q',title="Emisiones de CO2 equivalente (kt)"),
    x=alt.X('Año:O',title="",axis={"labelAngle":-90}),
    color='Clasificación GPC:N',
    #column='Nombre Región:N'

)

bars

## Plot de todas las regiones



In [ ]:
# Concatenar todos los datos

class_gpc = ["Energía estacionaria","Transporte","Residuos","IPPU","Agricultura","UTCUTS","Procesos industriales y uso de productos"]
zone_map = {'11BIO':"Biobío",'09MAU':"Maule",'10NUB':"Ñuble", '08OHI':"O'Higgins"}

df_regions = list()
for fdata in datos_zona:
    region = zone_map[fdata.name.split("_")[2]]
    gei_xls = pd.ExcelFile(fdata)
    gei_data = pd.read_excel(gei_xls, sheet_name=["GPC alcances serie CO2eq"],
                         index_col=[0,1],header=[0],skiprows=52,nrows=11)
    
    gei_data_f1 = gei_data["GPC alcances serie CO2eq"].iloc[1:,:-4]
    
    gei_data_f2 = gei_data_f1.loc[gei_data_f1["Clasificación GPC"].isin(class_gpc)]
    
    gei_tidy = gei_data_f2.melt(value_name="kt CO2 eq",var_name="Año",id_vars=["Clasificación GPC"],
        value_vars=gei_data_f2.columns[1:])
    
    #add region
    gei_tidy.loc[:,"Región"] = region
    
    #store DFs
    df_regions.append(gei_tidy)
    
tidy_final = pd.concat(df_regions)

In [ ]:
bars = alt.Chart(tidy_final).mark_bar(
    #cornerRadiusTopLeft=3,
    #cornerRadiusTopRight=3
).encode(
    y = alt.Y('kt CO2 eq:Q',title="Emisiones de CO2 equivalente (kt)"),
    x=alt.X('Año:O',title="",axis={"labelAngle":-90}),
    color='Clasificación GPC:N',
    #column='Nombre Región:N'

).facet(
    alt.Facet('Región:O',title=None),
    columns=2, center=True
).resolve_scale(
    y='independent',
    x='independent'
).properties(
    title="Emisiones y Absorciones de GEI (kt CO2 eq) de alcance 1 por sector, 1990-2018"
).configure_title(fontSize=16,anchor="middle")

#bars.save("emi_abs_macrozona_1990-2018.png",scale_factor=2.0)
bars